In [0]:
from gurobipy import *

In [0]:
import pandas as pd
import numpy as np

In [0]:
concat_2020 = pd.read_csv("concat_2020.csv")
concat_2020.head()

,Unnamed: 0,From Date,To Date,Week End Date,Retailer,Retailer Name,Game Number,Scratcher Sales,Street1,City,...,Max LT Winners,Top Prize,Top Prize Ct,Top Prizes Cashed,Validation Status,Launch Dt,End Dist Dt,End Valid Dt,Total Bins,Adj_Sales
0,0,11/24/2019,11/30/2019,11/30/2019,10009,SAFEWAY 0236,1134,1170,10926 W BELL RD,SUN CITY,...,4,"$1,750,000",6.0,5.0,2,12/4/2017,9/19/2027,9/19/2027,48,585.0
1,1,11/24/2019,11/30/2019,11/30/2019,10009,SAFEWAY 0236,1169,-10,10926 W BELL RD,SUN CITY,...,15,"$100,000",7.0,3.0,2,12/27/2018,5/1/2028,5/1/2028,48,-5.0
2,2,11/24/2019,11/30/2019,11/30/2019,10009,SAFEWAY 0236,1183,600,10926 W BELL RD,SUN CITY,...,7,"$500,000",6.0,5.0,2,12/3/2018,9/19/2028,9/19/2028,48,300.0
3,3,11/24/2019,11/30/2019,11/30/2019,10009,SAFEWAY 0236,1190,6600,10926 W BELL RD,SUN CITY,...,6,"$1,250,000",8.0,5.0,2,9/3/2018,7/7/2028,7/7/2028,48,3300.0
4,4,11/24/2019,11/30/2019,11/30/2019,10009,SAFEWAY 0236,1193,300,10926 W BELL RD,SUN CITY,...,40,"$10,000",15.0,15.0,2,5/6/2019,2/19/2020,8/18/2020,48,150.0


In [0]:
df_2020_by_price = concat_2020.pivot_table(index='Retailer', columns='Price', values='Adj_Sales', aggfunc=np.sum)

In [0]:
df_2020_by_price.fillna(0, inplace=True)
df_2020_by_price.head()

Price,$1,$10,$2,$20,$25,$3,$30,$5
Retailer,,,,,,,,
10009,3198.0,49630.0,8780.0,93900.0,4200.0,2874.0,31095.0,27497.5
10024,1365.0,19280.0,3254.0,20340.0,0.0,2100.0,7110.0,8095.0
10029,544.0,17410.0,3526.0,13640.0,0.0,618.0,7170.0,6600.0
10033,3048.0,71120.0,8716.0,78580.0,900.0,2346.0,53400.0,31280.0
10034,1916.0,18100.0,3714.0,21600.0,0.0,1188.0,16200.0,11090.0


In [0]:
df_2020_by_price_weekly = df_2020_by_price/14
df_2020_by_price_weekly.head()

Price,$1,$10,$2,$20,$25,$3,$30,$5
Retailer,,,,,,,,
10009,228.428571,3545.000000,627.142857,6707.142857,300.000000,205.285714,2221.071429,1964.107143
10024,97.500000,1377.142857,232.428571,1452.857143,0.000000,150.000000,507.857143,578.214286
10029,38.857143,1243.571429,251.857143,974.285714,0.000000,44.142857,512.142857,471.428571
10033,217.714286,5080.000000,622.571429,5612.857143,64.285714,167.571429,3814.285714,2234.285714
10034,136.857143,1292.857143,265.285714,1542.857143,0.000000,84.857143,1157.142857,792.142857


In [0]:
# df_2020_by_price_weekly['$1'] = df_2020_by_price_weekly['$1'].transform(lambda x: x/2)
# df_2020_by_price_weekly['$2'] = df_2020_by_price_weekly['$2'].transform(lambda x: x/5)
# df_2020_by_price_weekly['$3'] = df_2020_by_price_weekly['$3'].transform(lambda x: x/2)
# df_2020_by_price_weekly['$5'] = df_2020_by_price_weekly['$5'].transform(lambda x: x/6)
# df_2020_by_price_weekly['$10'] = df_2020_by_price_weekly['$10'].transform(lambda x: x/(5*2))
# df_2020_by_price_weekly['$20'] = df_2020_by_price_weekly['$20'].transform(lambda x: x/(2*2))
# df_2020_by_price_weekly['$30'] = df_2020_by_price_weekly['$30'].transform(lambda x: x/(2))
# df_2020_by_price_weekly['$25'] = df_2020_by_price_weekly['$25'].transform(lambda x: x)

In [0]:
#indexes

df = df_2020_by_price_weekly.copy()

df.shape

(2490, 8)

In [0]:
# get the average profit for each price category
df_price_payout = concat_2020[['Price','Payout %']].copy()
df_price_payout['Profit'] = df_price_payout['Payout %'].astype(str).map(lambda n: 1- float(n.strip('%'))/ 100.0)

In [0]:
df_price_payout = df_price_payout.groupby(['Price'])[['Profit']].mean()

In [0]:
df_price_payout

,Profit
Price,
$1,0.395033
$10,0.263546
$2,0.358206
$20,0.248656
$25,0.239700
$3,0.360606
$30,0.224863
$5,0.311325


In [0]:
# convert dataframe to a list
one_minus_Pi = df_price_payout['Profit'].values.tolist()
one_minus_Pi

[0.39503348890590934,
 0.2635463589372685,
 0.35820609269933185,
 0.2486563534689027,
 0.2397000000000061,
 0.36060591144802817,
 0.22486341172464575,
 0.3113250137315933]

In [0]:
df_2020_by_price_weekly.head()

Price,$1,$10,$2,$20,$25,$3,$30,$5
Retailer,,,,,,,,
10009,228.428571,3545.000000,627.142857,6707.142857,300.000000,205.285714,2221.071429,1964.107143
10024,97.500000,1377.142857,232.428571,1452.857143,0.000000,150.000000,507.857143,578.214286
10029,38.857143,1243.571429,251.857143,974.285714,0.000000,44.142857,512.142857,471.428571
10033,217.714286,5080.000000,622.571429,5612.857143,64.285714,167.571429,3814.285714,2234.285714
10034,136.857143,1292.857143,265.285714,1542.857143,0.000000,84.857143,1157.142857,792.142857


In [0]:
# get the Demand of i price category 
Di = df_2020_by_price_weekly.T

In [0]:
Di.head()

Retailer,10009,10024,10029,10033,10034,10037,10040,10046,10057,10062,...,66058,66061,66063,66065,66066,66067,66068,66073,66078,66079
Price,,,,,,,,,,,,,,,,,,,,,
$1,228.428571,97.500000,38.857143,217.714286,136.857143,89.485714,160.714286,291.428571,214.285714,146.642857,...,96.428571,0.000000,42.857143,42.857143,46.753247,96.428571,0.000000,85.714286,21.428571,21.428571
$10,3545.000000,1377.142857,1243.571429,5080.000000,1292.857143,459.428571,658.928571,3908.571429,2224.285714,1761.428571,...,737.142857,0.000000,1398.571429,788.571429,1122.077922,400.714286,171.428571,514.285714,557.142857,728.571429
$2,627.142857,232.428571,251.857143,622.571429,265.285714,160.457143,693.428571,1469.387755,810.571429,907.714286,...,493.071429,0.000000,514.285714,428.571429,420.779221,172.285714,0.000000,471.428571,114.000000,214.285714
$20,6707.142857,1452.857143,974.285714,5612.857143,1542.857143,990.857143,321.428571,4408.163265,1848.571429,771.428571,...,182.142857,771.428571,942.857143,685.714286,654.545455,321.428571,560.000000,342.857143,471.428571,385.714286
$25,300.000000,0.000000,0.000000,64.285714,0.000000,51.428571,0.000000,183.673469,85.714286,64.285714,...,0.000000,0.000000,42.857143,42.857143,46.753247,0.000000,0.000000,42.857143,0.000000,1.785714


In [0]:
# implement the function Min(C,Di), C refers to the capacity 
minC_Di = Di.applymap(lambda n: min(300, float(n)))
minC_Di.head()

Retailer,10009,10024,10029,10033,10034,10037,10040,10046,10057,10062,...,66058,66061,66063,66065,66066,66067,66068,66073,66078,66079
Price,,,,,,,,,,,,,,,,,,,,,
$1,228.428571,97.500000,38.857143,217.714286,136.857143,89.485714,160.714286,291.428571,214.285714,146.642857,...,96.428571,0.0,42.857143,42.857143,46.753247,96.428571,0.000000,85.714286,21.428571,21.428571
$10,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,...,300.000000,0.0,300.000000,300.000000,300.000000,300.000000,171.428571,300.000000,300.000000,300.000000
$2,300.000000,232.428571,251.857143,300.000000,265.285714,160.457143,300.000000,300.000000,300.000000,300.000000,...,300.000000,0.0,300.000000,300.000000,300.000000,172.285714,0.000000,300.000000,114.000000,214.285714
$20,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,...,182.142857,300.0,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
$25,300.000000,0.000000,0.000000,64.285714,0.000000,51.428571,0.000000,183.673469,85.714286,64.285714,...,0.000000,0.0,42.857143,42.857143,46.753247,0.000000,0.000000,42.857143,0.000000,1.785714


In [0]:
minC_Di = minC_Di.sum(axis=1)
minC_Di = minC_Di.values.tolist()
minC_Di

[290106.2531942304,
 706552.3778381117,
 655885.5412913858,
 693637.0760322115,
 157881.16167816386,
 522203.6146432222,
 599410.1366200439,
 717471.1088722466]

In [0]:
from scipy.optimize import minimize
import sys

def objective(x):
    obj = 0
    for i in range(8):
        obj -= x[i] * one_minus_Pi[i] * minC_Di[i]
    return obj

def constraint_equ(x):
    v = 24
    for i in x:
        v -= i
    return v
#     return 24 - sum(x)

# initial guess
n = 8
x0 = np.zeros(n)
x0[0] = 2.0
x0[1] = 5.0
x0[2] = 5.0
x0[3] = 3.0
x0[4] = 0.0
x0[5] = 2.0
x0[6] = 1.0
x0[7] = 6.0

# show initial objective
print('Initial Objective: ' + str(objective(x0)))

# optimize
b = [-sys.float_info.epsilon, 24.0]
bnds = (b, b, b, b, b, b, b, b)
con1 = {'type': 'eq', 'fun': constraint_equ} #for 24 bins in total constraint 
con2 = {'type':'eq', 'fun': lambda x : sum([x[i]-int(x[i]) for i in range(len(x))])} # for integer constraint

cons = ([con1, con2])
# cons = ([con1])
solution = minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)
x = solution.x

# show final objective
print('Final Objective: ' + str(objective(x)))

# print solution
print('Solution')
print('Num of $1 lottery: ' + str(x[0]))
print('Num of $10 lottery: ' + str(x[1]))
print('Num of $2 lottery: ' + str(x[2]))
print('Num of $20 lottery: ' + str(x[3]))
print('Num of $25 lottery: ' + str(x[4]))
print('Num of $3 lottery: ' + str(x[5]))
print('Num of $30 lottery: ' + str(x[6]))
print('Num of $5 lottery: ' + str(x[7]))

Initial Objective: -4703997.732597375
Final Objective: -4714122.314735303
Solution
Num of $1 lottery: 5.560254438037473
Num of $10 lottery: 3.1846548000414704e-08
Num of $2 lottery: 17.12164496695987
Num of $20 lottery: 2.2357573675077805e-08
Num of $25 lottery: 1.4255187725676353
Num of $3 lottery: 0.0019297100984099415
Num of $30 lottery: 3.8527381599511785e-08
Num of $5 lottery: 2.9421559753701456e-08


In [0]:
print('Num of $1 lottery: ', int(round(x[0])))
print('Num of $10 lottery: ', int(round(x[1])))
print('Num of $2 lottery: ', int(round(x[2])))
print('Num of $20 lottery: ', int(round(x[3])))
print('Num of $25 lottery: ', int(round(x[4])))
print('Num of $3 lottery: ', int(round(x[5])))
print('Num of $30 lottery: ', int(round(x[6])))
print('Num of $5 lottery: ', int(round(x[7])))

Num of $1 lottery:  6
Num of $10 lottery:  0
Num of $2 lottery:  17
Num of $20 lottery:  0
Num of $25 lottery:  1
Num of $3 lottery:  0
Num of $30 lottery:  0
Num of $5 lottery:  0
